### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_report, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220711_scenario_projections.ipynb")

2022/07/12 15:13:29 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [53]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220628_july_gov_briefing/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# fit the model
model = do_single_fit(**fit_args, **model_args)

2022/07/12 16:58:47 INFO|-------------------------|{} Retrieving vaccinations data
2022/07/12 16:58:53 INFO|-------------------------|{} Constructing vaccination projections
2022/07/12 16:58:53 INFO|-------------------------|{} Retrieving hospitalizations data
2022/07/12 16:58:55 INFO|----------------------|{} Prepping Model
2022/07/12 16:58:55 INFO|-----------------------|{} Prepping Model
2022/07/12 16:59:56 INFO|----------------------|{} Model prepped for fitting in 61.11455337500047 seconds.
2022/07/12 16:59:56 INFO|----------------------|{} Will fit 25 times
2022/07/12 17:02:47 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 170.85787866700048 seconds: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5380334941579741}, 42: {'co': 0.0005409486889829486}, 56: {'co': 0.6816118768777373}, 70: {'co': 0.89119421452826}}
2022/07/12 17:02:47 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/07/1

2022/07/12 17:07:31 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 10.59446970800036 seconds: {504: {'co': 0.797010200074116}, 518: {'co': 0.7916107291516231}, 532: {'co': 0.7704198734916661}, 546: {'co': 0.7250135850840854}, 560: {'co': 0.7573723232428697}, 574: {'co': 0.7526775409964072}}
2022/07/12 17:07:31 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/07/12 17:07:43 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 11.361709583000447 seconds: {532: {'co': 0.770267911901516}, 546: {'co': 0.7266770720143305}, 560: {'co': 0.753794663735644}, 574: {'co': 0.7628554945270086}, 588: {'co': 0.798926122154828}, 602: {'co': 0.7800606093975648}}
2022/07/12 17:07:43 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/07/12 17:07:52 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 7.747266875000605 seconds: {560:

In [54]:
model.spec_id

3107

### Create and run scenarios from Feb 2022 to present

In [55]:
multiprocess = None

scenario_params = json.load(open('covid_model/analysis/20220628_july_gov_briefing/scenario_params.json'))

model_args = {
    'base_spec_id': 3101 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.5, 0.7, 0.9]: # 50%, 70% , or 90% for-COVID
    for school_contact_increase in [1.0, 1.1, 1.2]:
        for immune_escape_sens in [1.0, 1.05, 0.95]:
        #for (weak_escape, strong_escape) in [(0.75, 0.1), (0.8, 0.2)]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
            school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},"desc": "After August 15th, beta increases by a factor of the school contact increase"}]
            lt5_vacc_adjust = [{"param": "immunity", "attrs": {'age': '0-19', 'vacc': 'shot1'}, "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5, "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5}}]
            immune_sens = [{"param": "immune_escape", "from_attrs":{"variant": ["omicron", "ba2", "ba2121"]}, "to_attrs": {"variant": ["ba45"]}, "mults": {"2020-01-01": immune_escape_sens},  "desc": "sensitivity analysis for immune escape"}]
            scenario_model_args.append({'params_defs': scenario_params + school_contact + lt5_vacc_adjust + immune_sens,
                                       'hosp_reporting_frac': hrf, # hospital reporting fraction is not specified in params.json, it's a separate model argument that we created (i.e. we didn't identify a "param" in the dictionary)
                                       'tags': {'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase, 'immune_escape_sens': immune_escape_sens}})
            
            

In [56]:
# determine the number of parameters used
len(scenario_args_list)

27

In [57]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/07/12 17:11:06 INFO|----------------------------|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_sens': 1.0} Retrieving vaccinations data
2022/07/12 17:11:07 INFO|----------------------------|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_sens': 1.0} Constructing vaccination projections
2022/07/12 17:11:07 INFO|----------------------------|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_sens': 1.0} Retrieving hospitalizations data
2022/07/12 17:11:09 INFO|-------------------------|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_sens': 1.0} Prepping Model
2022/07/12 17:11:09 INFO|--------------------------|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_sens': 1.0} Prepping Model
2022/07/12 17:12:06 INFO|-------------------------|{'hosp_frac': 0.5, 'school_contact_increase': 1.0, 'immune_escape_sens': 1.0} Model prepped for fitting in 57.26741566700002 seconds.
2022/07/12 17:12:06 INFO|------

ValueError: cannot convert float NaN to integer

### Run the report for each fit model

In [ ]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

In [ ]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

In [ ]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

In [ ]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')